___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data and Setup

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales', axis=1)
y = df['sales']

### Polynomial Conversion

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [6]:
poly_features = polynomial_converter.fit_transform(X)

poly_features.shape

(200, 19)

### Train | Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

X_train.shape

(140, 19)

----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
# help(StandardScaler)

In [12]:
scaler = StandardScaler()

In [13]:
scaler.fit(X_train)

StandardScaler()

In [14]:
# only X transformed
X_train = scaler.transform(X_train) # you can name scaled_X_train
X_test = scaler.transform(X_test) # you can name scaled_X_test

In [15]:
# compare
poly_features[0], X_train[0]

(array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
        8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
        4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
        3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
        9.88757280e+04, 1.81010592e+05, 3.31373888e+05]),
 array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
         1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
        -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
        -0.66096022, -0.16360242,  0.54694754,  1.37075536]))

## Ridge Regression

Make sure to view video lectures for full explanation of Ridge Regression and choosing an alpha.

In [16]:
from sklearn.linear_model import Ridge

In [18]:
# alpha is lambda
ridge_model = Ridge(alpha=10)

In [19]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [20]:
test_predictions = ridge_model.predict(X_test)

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

# the higher the better
MAE, MSE, RMSE

(0.5774404204714167, 0.8003783071528353, 0.894638646131965)

How did it perform on the training set? (This will be used later on for comparison)

In [23]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_predictions)
MAE

0.5288348183025304

### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [32]:
# sklearn also have another Ridge model called RidgeCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import SCORERS

In [35]:
SCORERS.keys()
# the higher the better

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [28]:
# help(RidgeCV)

In [25]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')

In [26]:
# The more alpha options you pass, the longer this will take.
# Fortunately our data set is still pretty small
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [29]:
ridge_cv_model.alpha_

0.1

In [30]:
test_predictions = ridge_cv_model.predict(X_test)

In [31]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

MAE, MSE, RMSE

(0.4273774884351013, 0.3820129881538421, 0.6180719926948981)

In [36]:
# Training Set Performance
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.30941321056569865

In [37]:
# for L2 none of the coefficients are zero
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])


-----

## Lasso Regression

In [38]:
from sklearn.linear_model import LassoCV # or Lasso

In [39]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [40]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [41]:
lasso_cv_model.alpha_

0.4943070909225832

In [42]:
test_predictions = lasso_cv_model.predict(X_test)

In [43]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

MAE, MSE, RMSE
# Lasso is not as good as Ridge (in this case)

(0.6541723161252867, 1.2787088713079886, 1.1308001022762548)

In [44]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.6912807140820709

In [45]:
lasso_cv_model.coef_

# a lot of coefficients are zero (model complexity reduced)

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [46]:
from sklearn.linear_model import ElasticNetCV

In [47]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], tol=0.01)

In [48]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [51]:
# put _ at the end to get the best value
elastic_model.l1_ratio_

1.0

In [52]:
test_predictions = elastic_model.predict(X_test)

In [53]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

MAE, MSE, RMSE

(0.5663262117569452, 0.560334021463884, 0.7485546215633726)

In [54]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.43075829904723684

In [55]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])

-----
---